# Stock Prediction Determiner - Model Development

This notebook trains and saves LSTM and Linear Regression models for stock price prediction.

In [ ]:
# Import libraries
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import joblib
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Create model directory
os.makedirs('model', exist_ok=True)

In [ ]:
# Download stock data
df = yf.download('AAPL', start='2015-01-01', end='2023-01-01')
df = df[['Close']]
df.head()

## Prepare Data for LSTM

In [ ]:
# Normalize data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# Save scaler
joblib.dump(scaler, 'model/scaler.pkl')

In [ ]:
# Create dataset for LSTM
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i])
        y.append(data[i])
    return np.array(X), np.array(y)

X, y = create_dataset(scaled_data)
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
# Split into train and test
split = int(len(X)*0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

## Build and Train LSTM Model

In [ ]:
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    LSTM(50),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=5, batch_size=32)

In [ ]:
# Save LSTM model
model.save('model/lstm_model.h5')

## Prepare Data and Train Linear Regression Model

In [ ]:
# Prepare data with lag feature
df['Prev_Close'] = df['Close'].shift(1)
df = df.dropna()

X_lr = df[['Prev_Close']].values
y_lr = df['Close'].values

X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_lr, y_lr, test_size=0.2, random_state=42)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train_lr, y_train_lr)

In [ ]:
# Save linear regression model
joblib.dump(lr_model, 'model/linear_model.pkl')

## Plot Actual vs Predicted for LSTM and Linear Regression (Optional)

In [ ]:
# LSTM Predictions
lstm_pred_scaled = model.predict(X_test)
lstm_pred = scaler.inverse_transform(lstm_pred_scaled)
y_test_actual = scaler.inverse_transform(y_test)

# Linear Regression Predictions
lr_pred = lr_model.predict(X_test_lr)

import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
plt.plot(y_test_actual, label='Actual Close Price (LSTM)')
plt.plot(lstm_pred, label='Predicted Close Price (LSTM)')
plt.legend()
plt.title('LSTM Model Prediction')
plt.show()

plt.figure(figsize=(12,5))
plt.plot(y_test_lr, label='Actual Close Price (Linear Regression)')
plt.plot(lr_pred, label='Predicted Close Price (Linear Regression)')
plt.legend()
plt.title('Linear Regression Prediction')
plt.show()